In [7]:
import sys
import pygame

# -------------------------
# Game settings
# -------------------------
NUM_DISKS = 3
PEG_COUNT = 3
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 500
PEG_WIDTH = 10
PEG_HEIGHT = 200
BASE_HEIGHT = 20
DISK_HEIGHT = 20
PEG_SPACING = SCREEN_WIDTH // (PEG_COUNT + 1)

# Colors
BACKGROUND_COLOR = (173, 216, 230)
PEG_COLOR = (139, 69, 19)
DISK_COLORS = [
    (255, 105, 97),
    (144, 238, 144),
    (100, 149, 237),
    (238, 130, 238),
    (255, 215, 0),
]

pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Tower of Hanoi")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 32)

# -------------------------
# Initial game state
# -------------------------
def reset_game():
    return [list(range(NUM_DISKS, 0, -1)), [], []], None, 0, False

pegs, held_disk, move_count, game_won = reset_game()
reset_button = None
retry_button = None

# -------------------------
# Helper functions
# -------------------------
def draw_button(text, center, color=(0, 128, 0)):
    button_font = pygame.font.SysFont(None, 28)
    button_text = button_font.render(text, True, (255, 255, 255))
    text_rect = button_text.get_rect(center=center)
    button_rect = pygame.Rect(
        text_rect.x - 10,
        text_rect.y - 5,
        text_rect.width + 20,
        text_rect.height + 10
    )
    pygame.draw.rect(screen, color, button_rect, border_radius=5)
    screen.blit(button_text, text_rect)
    return button_rect

def draw_game():
    screen.fill(BACKGROUND_COLOR)

    # Draw pegs and bases
    for i in range(PEG_COUNT):
        peg_x = PEG_SPACING * (i + 1)
        peg_y = SCREEN_HEIGHT - BASE_HEIGHT - 40
        pygame.draw.rect(
            screen, PEG_COLOR,
            (peg_x - PEG_WIDTH // 2, peg_y - PEG_HEIGHT, PEG_WIDTH, PEG_HEIGHT)
        )
        pygame.draw.rect(
            screen, PEG_COLOR,
            (peg_x - 60, peg_y, 120, BASE_HEIGHT)
        )

    # -------------------------
    # Draw disks (FIXED)
    # -------------------------
    for peg_index, peg in enumerate(pegs):
        peg_x = PEG_SPACING * (peg_index + 1)
        for level, disk_size in enumerate(peg):
            width = disk_size * 30
            disk_color = DISK_COLORS[(disk_size - 1) % len(DISK_COLORS)]
            disk_x = peg_x - width // 2

            # FIXED Y POSITION
            disk_y = (
                SCREEN_HEIGHT
                - BASE_HEIGHT
                - 40
                - (level + 1) * DISK_HEIGHT
            )


            pygame.draw.rect(
                screen,
                disk_color,
                (disk_x, disk_y, width, DISK_HEIGHT)
            )

    # Draw held disk
    if held_disk:
        disk_size, _ = held_disk
        width = disk_size * 30
        disk_color = DISK_COLORS[(disk_size - 1) % len(DISK_COLORS)]
        mouse_x, mouse_y = pygame.mouse.get_pos()
        pygame.draw.rect(
            screen,
            disk_color,
            (mouse_x - width // 2, mouse_y - DISK_HEIGHT // 2, width, DISK_HEIGHT)
        )

    # Peg labels
    for i in range(PEG_COUNT):
        peg_x = PEG_SPACING * (i + 1)
        label_text = font.render(f"Tower {i + 1}", True, (0, 0, 0))
        label_rect = label_text.get_rect(center=(peg_x, SCREEN_HEIGHT - 10))
        screen.blit(label_text, label_rect)

    # Move counter
    move_text = font.render(f"Moves: {move_count}", True, (0, 0, 0))
    min_moves = (2 ** NUM_DISKS) - 1
    min_text = font.render(f"Min: {min_moves}", True, (0, 0, 0))
    screen.blit(move_text, (20, 20))
    screen.blit(min_text, (200, 20))

    global reset_button, retry_button
    reset_button = draw_button("Reset", (SCREEN_WIDTH - 80, 30))

    retry_button = None
    if game_won:
        win_text = font.render(
            f"You Win! Total Moves: {move_count}", True, (0, 0, 0)
        )
        text_rect = win_text.get_rect(center=(SCREEN_WIDTH // 2, 60))
        screen.blit(win_text, text_rect)
        retry_button = draw_button(
            "Play Again", (SCREEN_WIDTH // 2, 120), color=(0, 100, 200)
        )

    pygame.display.flip()

def get_peg_at_position(pos):
    x, _ = pos
    for i in range(PEG_COUNT):
        peg_x = PEG_SPACING * (i + 1)
        if abs(x - peg_x) < 60:
            return i
    return None

def is_move_valid(disk_size, target_peg_index):
    if not pegs[target_peg_index]:
        return True
    return pegs[target_peg_index][-1] > disk_size

# -------------------------
# Game loop
# -------------------------
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            if reset_button and reset_button.collidepoint(event.pos):
                pegs, held_disk, move_count, game_won = reset_game()

            elif retry_button and retry_button.collidepoint(event.pos):
                pegs, held_disk, move_count, game_won = reset_game()

            elif not game_won:
                peg_clicked = get_peg_at_position(event.pos)
                if peg_clicked is not None:
                    if held_disk is None:
                        if pegs[peg_clicked]:
                            disk_size = pegs[peg_clicked].pop()
                            held_disk = (disk_size, peg_clicked)
                    else:
                        disk_size, from_peg = held_disk
                        if is_move_valid(disk_size, peg_clicked):
                            pegs[peg_clicked].append(disk_size)
                            move_count += 1
                            held_disk = None
                        else:
                            pegs[from_peg].append(disk_size)
                            held_disk = None

    if len(pegs[-1]) == NUM_DISKS and not game_won:
        game_won = True

    draw_game()
    clock.tick(60)

pygame.quit()
